In [ ]:
import os
import matplotlib.pyplot as plt
from PencilDrawingBySketchAndTone import *

In [ ]:
def gen_pencil_drawing_v2(img, kernel_size, stroke_width=0, num_of_directions=8, smooth_kernel="gauss",
                       gradient_method=0, rgb=False, w_group=0, pencil_texture_path="", stroke_darkness=1, tone_darkness=1):
    if not rgb:
        # Grayscale image:
        im = img
    else:
        # RGB image:
        yuv_img = color.rgb2yuv(img)
        im = yuv_img[:,:,0]
    # Generate the Stroke Map:
    S = gen_stroke_map(im, kernel_size, stroke_width=stroke_width, num_of_directions=num_of_directions,
                       smooth_kernel=smooth_kernel, gradient_method=gradient_method)
    S = np.power(S, stroke_darkness)
    # Generate the Tone Map:
    J = gen_tone_map(im, w_group=w_group)
    
    # Read the pencil texture:
    if not pencil_texture_path:
        pencil_texture = io.imread('drawing-patterns/drawing-pattern-I.jpg', as_gray=True)
    else:
        pencil_texture = io.imread(pencil_texture_path, as_gray=True)
    # Generate the Pencil Texture Map:
    T = gen_pencil_texture(im, pencil_texture, J)
    T = np.power(T, tone_darkness)
    # The final Y channel:
    R = np.multiply(S, T)
    
    if not rgb:
        return R
    else:
        yuv_img[:,:,0] = R
        return S,J,T,R,exposure.rescale_intensity(color.yuv2rgb(yuv_img), in_range=(0, 1))

directory = 'path/to/imagenet/val/'
classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
image_locations = []
for cls in classes:
    image_locations += sorted([os.path.join(directory,cls,entry.name) for entry in os.scandir(directory+'/'+cls)])

os.makedirs('../examples_drawing', exist_ok=True)
for idx in [8*50, 9*50+1, 10*50+7, 234*50+5, 779*50+1, 786*50+3, 790*50+5, 794*50+7,
            235*50+1, 802*50+0, 803*50+8, 808*50+9, 809*50+2, 814*50+5, 891*50+5, 909*50+0]:
    load_path = image_locations[idx]
    pencil_tex = 'drawing-patterns/drawing-pattern-I.jpg'
    img_pencil = cv2.cvtColor(cv2.imread(pencil_tex), cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(cv2.imread(load_path), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_AREA)
    S,J,T,R,img_drawing = gen_pencil_drawing_v2(img, kernel_size=8, stroke_width=1, num_of_directions=8, smooth_kernel="gauss",
                       gradient_method=1, rgb=True, w_group=2, pencil_texture_path=pencil_tex,
                               stroke_darkness=2, tone_darkness=1.5)
    fig, axs = plt.subplots(1,7,figsize=(15,5))
    extension = image_locations[idx].split('.')[-1]
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet.{extension}', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    axs[0].imshow(img)
    axs[0].axis('off')
    axs[0].set_title('Original Image')
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet_S.{extension}', S*255)
    axs[1].imshow(S, cmap='gray')
    axs[1].axis('off')
    axs[1].set_title('S - Pencil Stroke')
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet_J.{extension}', J*255)
    axs[2].imshow(J, cmap='gray')
    axs[2].axis('off')
    axs[2].set_title('J - Tonal Map')
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet_T.{extension}', T*255)
    axs[3].imshow(T, cmap='gray')
    axs[3].axis('off')
    axs[3].set_title('T - Tonal Texture')
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet_R.{extension}', R*255)
    axs[4].imshow(R, cmap='gray')
    axs[4].axis('off')
    axs[4].set_title('R - Pencil Sketch')
    cv2.imwrite(f'../examples_drawing/id_{idx}_imagenet_drawing.{extension}', cv2.cvtColor(np.clip(img_drawing*255, 0, 255).astype(np.uint8),cv2.COLOR_RGB2BGR))
    axs[5].imshow(img_drawing)
    axs[5].axis('off')
    axs[5].set_title('Final Image')
    img_drawing_pattern = cv2.resize(img_pencil, (img_pencil.shape[1], img_pencil.shape[0]), interpolation = cv2.INTER_AREA)
    cv2.imwrite(f'../examples_drawing/id_{idx}_drawing_pattern.{extension}', img_drawing_pattern)
    axs[6].imshow(img_drawing_pattern)
    axs[6].axis('off')
    axs[6].set_title('Drawing Pattern')